In [ ]:
%pip install -r requirements.txt

In [2]:
from control2048 import Control
from game import GameOf2048
import numpy as np
from random import choices, seed

In [3]:
class DTAgent:
    # tries to do the move with the best expected best expected score.
    # since the state space is so huge,
    # we look ahead to some max depth.
    # the depth changes depending on how many tiles are numbered.
    # less tiles -> less risk of losing -> less lookahead to survive
    # more tiles -> less future states to look at -> more depth (or breadth)
    # analyzed in the same time


    def __init__(self):
        self.cache = {}
        self.moves = 0
    
    def nextMove(self, board):
        # we assume the board can be moved
        moves = []
        for move in range(4):
            self.game = GameOf2048()
            # moves are ints,
            # this is explained in the GameOf2048 class definition

            moved, _ = self.game.transform(board, move)
            # numTiles = len(list(self.game.availableTiles(moved)))
            depth = 2 # min(14//numTiles+1, 4)

            if not np.array_equal(board, moved):
                moves.append((move, self.expectedPointsFrom(moved, depth)))
            else:
                pass # not gonna use useless moves
        
        moves.sort(key=lambda x: x[1])
        return moves[-1][0]


    def expectedPointsFrom(self, board, depth):
        key = str(board)
        if key in self.cache:
            return self.cache[key]
        
        lost = self.game.verifyLoss(board)
        if depth == 0 or lost:
            self.cache[str(board)] = 0
            return 0
        # else...

        score = 0

        for move in range(4):
            moved, points = self.game.transform(board, move)

            if np.array_equal(board, moved):
                continue # nothing to do, useless move
            else:
                tiles = list(self.game.availableTiles(moved))
                samples = min(len(tiles),3)

                for coord in choices(tiles, k=samples):
                    x, y = coord
                    
                    moved[x,y] = 4
                    score += points+self.expectedPointsFrom(moved, depth-1)

                    moved[x,y] = 2
                    score += 9*(points+self.expectedPointsFrom(moved, depth-1))
                    
                    moved[x,y] = 0
        
        self.cache[str(board)] = score

        return score

[[0 0 0 0]
 [0 0 0 2]
 [0 0 0 0]
 [0 0 0 2]]

[[0 0 0 4]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [5]:
env = GameOf2048()
agent = DTAgent()

seed(0)
points = []
steps = []
lost = []
maxTile = []

for _ in range(1):
    env.reset()
    
    total = 0
    
    stepAdded = False
    for step in range(1000):
        if env.lost:
            steps.append(step+1)
            stepAdded=True
            break
        total += env.step(agent.nextMove(env.currentBoard))[1]
        
    if not stepAdded:
        steps.append(step+1)
    points.append(total)
    lost.append(env.lost)
    maxTile.append(np.max(env.currentBoard))

[[0 2 0 2]
 [0 0 0 2]
 [0 0 0 0]
 [0 0 0 0]]


In [ ]:
min(points),max(points),np.mean(points),np.median(points)

(0, 744.0, 7.44, 0.0)

In [ ]:
len([i for i in lost if i])/len(lost)

1.0

In [ ]:
max(maxTile)

64

In [ ]:
max(steps)

100